# The `swifttools.ukssdc.query.SXPS` class

The `SXPSQuery` class is a child class of the [`swifttools.ukssdc.query` class](../query.ipynb) extending its functionality to give some GRB-specific options. It makes use of the [`swifttools.ukssdc.data.SXPS` module](../data/SXPS.ipynb) to allow you to download SXPS data products for objects found by querying.

In this guide I am going to cover the SXPS-specific query features and show you some examples of how to get data, but I am not going into all the details of the generic query syntax, or the product access functions. For those I refer you to the [`query`](../query.ipynb) and [`data.SXPS`](../data/SXPS.ipynb) documentation.

First, let's import the module, using our normal shortform:

In [ ]:
import swifttools.ukssdc.query as uq

## Page contents

* [The `SXPSQuery` class](#sxpsquery)
* [SXPS products](#prods)
    * [Sources](#sources)
      * [Source Details](#sinfo)
      * [Observation lists](#sobs)
      * [Light curves](#slc)
      * [Spectra](#sspec)
      * [Images](#sim)
      * [Vizier and SIMBAD matches](#scat)
      * [XRTProductRequests](#sxpr)
      
    * [Datasets](#datasets)
      * [Dataset Details](#dsinfo)
      * [Images](#dsim)
    * [Transients](#trans)
      * [Transient Details](#tinfo)
      * [Light curves](#tlc)
      * [Spectra](#tspec)
      * [Images](#tim)
      * [Vizier and SIMBAD matches](#tcat)
      * [XRTProductRequests](#txpr)
    * [Full table downloads](#fulltab)

<a id='sxpsquery'></a>
## The `SXPSQuery` class

To execute queries on the SXPS catalogues, we need an object from the `SXPSQuery` class, so let's start off by making one and, as usual, turning `silent` mode off for this demo:

In [ ]:
q = uq.SXPSQuery(silent=False)

The first thing we will need to do is decide what we want to query. Using this class we have multiple catalogues that we can query (2SXPS and LSXPS at the time of writing) and each of these has multiple tables. As you will know if you've read the [`ObsQuery`](../query.ipynb) and [`GRBQuery`](GRB.ipynb) documentation, we can view/set selected catlogue and table with the `cat` and `table` variables and can find out what options exist with `cats` and `tables`, so:

In [ ]:
print(f"Selected catalogue: {q.cat}")
print(f"Selected table: {q.table}\n")
print(f"Available catalogues: {q.cats}")
print(f"Available tables in this catalogue: {q.tables}\n")

The table and catalogue can be specified in the constructor, or by setting the relevant variables, for example:

In [ ]:
q = uq.SXPSQuery(silent=False, cat="LSXPS", table="datasets")
q.cat = "2SXPS"
q.table = "sources"

Personally, I would recommend *always* setting the catalogue and table explicitly in the constructor, to make your code's operation more readable and in case for some reason the defaults ever change (I do not intend doing this, but maybe one day I'll go over to the Dark Side).

There is also an SXPS-specific feature: table subsets. That is, some tables have predefined subsets which you can select to search: if you've used the web interface you should be familiar with these; they are defined and documented in the [online documentatation](https://www.swift.ac.uk/LSXPS/docs.php#Table2) and [LSXPS paper](https://www.swift.ac.uk/LSXPS/paper.php). At the time of writing only the sources and datasets tables have such subsets, and the options are "clean" and "ultraclean", but you can check the available subsets for the selected catalogue and table using the `subsets` variable:

In [ ]:
q.subsets

And you can view / set the subset using the `q.subset` variable:

In [ ]:
print(f"The current subset is {q.subset}")
q.subset = "Clean"
print(f"Now it is {q.subset}")

And you can unset it as well:

In [ ]:
q.subset = None
print(f"Now it is {q.subset}")

One small note here: this `subset` variable belonging to the `SXPSQuery` class, should not be confused with the [`subsets` argument](../query.ipynb#subsets) that you can pass to the various [functions to get products](#prods). In this case here, `subsets` is a *property of the catalogue* and relates to a predefined subset of catalogue results, so your query is on this catalogue subset. In the latter case, which we will encounter below, 
we are telling a function that gets products for our query results to return products for a subset *of the results we have already obtained* (the reuse of name is unfortunate, but a sensible alternative has failed to present itself to my mind. No doubt one will occur 13.2 seconds after release of this module).

So, let's put all of this together and make an example query, just to remind you of the syntax. I am going to request a cone search to find all datasets near GK Per, with at least 5ks of exposure.

In [ ]:
q = uq.SXPSQuery(silent=False, cat="LSXPS", table="datasets")
q.addConeSearch(name="GK Per", radius=12, units="arcmin")
q.addFilter(["ExposureUsed", ">=", 5000])
q.submit()
q.results

For more details about how to forumulate an execute a query you need to read the [top-level query documentation](../query.md).

<a id='prods'></a>

### SXPS Products

Having carried out a query, such as above, you may want to access some of the data related to the objects found. You could take the object identifiers from your queries, and then pass them to the [`swifttools.ukssdc.data.SXPS` functions](../data/SXPS.ipynb) to get at your data, but this is a bit of a drag, so the `SXPSQuery` module provides wrappers to do all of that for you, as we will now demonstrate. Note: I am *not* going to detail all of the arguments for the various data access functions, nor what they return: read the [data documentation](../data/SXPS.ipynb) for that. I will, however, give a few quick notes here (as I did for GRBs):

First, let me remind you that by default all of the functions to get data (starting `get`), when called via the `query` module, neither save data to disk nor return it, but save it in a variable inside your `SXPSQuery` object. You can change this behaviour with the `saveData` and `returnData` arguments, but even then, the data will still be stored in class variables.

Second: the data you request must be indexed in some way (to show which object they belong to). In the [`data.SXPS` module](../data/SXPS.ipynb) you controlled how the results were indexed by how you identified the objects you wanted, i.e. whether you supplied the `sourceName` or `sourceID` parameter to a `get()` function. When getting data from an `SXPSQuery` object, you use the `byName` or `byID` parameter to state how you want the results indexed (or, for datasets, `byObsID` and `byDatasetID`). We will see some examples below.

Third: You can request products for only a subset of the results your query found, using the [`subsets` argument](../query.ipynb#subsets).

So, the basic syntax of every product retrieval function is the same:

`q.get<something>(byID, byName, subset, returnData, saveData, **kwargs)`

where `**kwargs` are any arguments you want to pass to the underlying function in [`SXPS.GRB`](../data/SXPS.ipynb).

So, without wasting more time on abstract explanations, let's get to the examples. Data products are only available for the 'sources', 'datasets' and 'transients' tables, and we'll go through these in turn and (hopefully) at speed.

---

<a id='sources'></a>
## Sources

All of the source products we looked at in the [`swifttools.ukssdc.data.SXPS` documentation](../data/SXPS.ipynb) are available for us and the function names are almost identical. First of all, let's run a query to get us some sources:

In [ ]:
q = uq.SXPSQuery(cat="LSXPS", table="sources", silent=False)
q.addConeSearch(name="GRB 060729", radius=2, units="arcmin")
q.addFilter(("DetFlag", "=", 0))
q.submit()

At the time of writing, this found me 6 sources, although given that LSXPS is dynamic, this could change! If you turned off silent mode, then you can see how many rows you had like this:

In [ ]:
q.numRows

To speed up the processing below, and to remind you how to get subsets, I'm going to make a subset that has only results less than 2' off axis, and with a positive HR1 value.

In [ ]:
mySS = (q.results["MeanOffAxisAngle"] < 2) | (q.results["HR1"] > 0)

As a quick aside, [filters in query expressions](../query.ipynb#advanced) are always combined with an 'AND', so the above filter can *only* be done by making a subset of the query results.

Now we have a query, with some results, let's get some products.

<a id='sinfo'></a>
### Source details

**Note: This functionality only exists for LSXPS**

The `SXPSQuery` class provides two ways of getting the source details. The 'obvious' one is the `getsourceDetails()` function which wraps [the equivalent function in the `data` module](../data/SXPS.ipynb#sinfo), but since your `SXPSQuery` object already knows which table you have been querying, you can save your fingers and just call a function `getDetails()`. This will silently redirect to `getSourceDetails()` (or one of the other `Details` functions) according to the selected table.

Really, it's up to you which you use. `getsourceDetails()` is 'safer' in that you can tell at a glance at the code what is being called, and if you accidentally call the function with the wrong table selected, you will get an error rather than a load of data you didn't want. `getDetails()` on the other hand is much more flexible, especially if you're calling it inside a function or loop where you don't know at the time of coding which table is going to be passed in.

So, below I will use both forms, just to make the point:

In [ ]:
q.getSourceDetails(byName=True, subset=mySS)

In [ ]:
q.sourceDetails.keys()

As I warned, this function didn't return anything, because the data are instead stored in the `q.sourceDetails` variable. If for some reason, you really want to return the data, and can't bring yourself to access this variable, you can pass the argument `returnData=True` to the function, but in doing so, know that I judge you.

The `q.sourceDetails dict` is indexed by object name, because I have the `byName=True` argument.

One nice thing about the internal variable of products is that it can be updated. Imagine that you realiased that you gave the wrong subset, and wanted to get more results: just rerun the function the the new data will be added to the `q.sourceDetails dict`.

In [ ]:
q.getDetails(byName=True, subset=mySS)
q.sourceDetails.keys()

As you see, the extra data have been added to the `dict`. You will also note that even though I used the shortened `getDetails()` function the result still got (correctly) directed to the `sourceDetails` variable.


If this is not what you wanted, you can always 'forget' the data you had first, using the generic `clearProduct()` function:

In [ ]:
q.clearProduct("sourceDetails")
print(f"sourceDetails is: {q.sourceDetails}\n\n")
q.getDetails(byID=True, subset=mySS)
q.sourceDetails.keys()

Here I cleared the variable, and then sent another request, and, just to show that you can, ask for these results to be indexed by ID, not name.

<a id='sobs'></a>
### Source observations lists

**Note: This functionality only exists for LSXPS**

We can also get the list of observations and targetIDs covering the source(s) from our query, using the `getObsList()` function, which saves its data in the `sourceObsList` class variable.

**Important note** internally this uses `getSourceDetails()`, and to save time, if you've already got the source details for your objects, it will not repeat the look up. But if this is the case, then the `byName` or `byID` argument you give to `getObsList()` will be ignored, and whatever you used for `getSourceDetails()` used instead.

If you don't follow that, just clear the sourceDetails first:

In [ ]:
q.clearProduct("sourceDetails")

q.getObsList(byName=True, subset=mySS, useObs="allDet")

q.sourceObsList

Again, for full details of the arguments, see the [function in the `data.SXPS` module](../data/SXPS.ipynb#sobs).

If you wanted to download theese data using the [main `data` module](../data.ipynb) you could do something like below:

In [ ]:
import swifttools.ukssdc.data as ud

ud.downloadObsData(
    q.sourceObsList["LSXPS J062126.9-622317"]["obsList"],
    instruments=("xrt",),
    destDir="/tmp/APIDemo_SXPSq_data",
    silent=False,
)

In principle I could provide some wrapper function in the `query.SXPS` class to do this for you. I've decided against this for now, partly because there are a few little complexities to manage, and partly because I could continue forever adding little extras and never release this API. Honesty, I can't imagine this option being in demand. If you really want this functionality, let me know and I'll consider adding it.

<a id='slc'></a>
### Source light curves.

If you read the above, then the mechanism for getting source light curves is going to come as a huge shock. Brace yourself:

In [ ]:
q.getLightCurves(
    byID=True,
    subset=q.results["MeanOffAxisAngle"] < 2,
    timeFormat="MJD",
    binning="obs",
)

And, as I didn't give `saveData=True` the results have only been stored in a class variable, `lightCurves`:

In [ ]:
q.lightCurves.keys()

There are very many arguments related to the SXPS light curves, and they were [discussed in nauseating detail elsewhere](../data/SXPS.ipynb#lightcurves) so I won't repeat myself here.

As with `sourceDetails` and everything else, you can repeat this call with a different (or no) subset and it will simply add those sources to the `q.lightCurves` `dict`. You can also change some of the arguments for `getLightCurves)`, if you decide you wanted to get a different band, for example, and these will be added in, as I'll show in a second.

**BUT there is a warning** Once you've saved some light curves, any subsequent downloads must have the *same `binning` and `timeFormat`* arguments as the first call (the reasons for this are too boring to document here. You can email me if you care). So the following cell will produce an error:

In [ ]:
q.getLightCurves(
    byID=True,
    subset=q.results["MeanOffAxisAngle"] > 2,
    timeFormat="TDB",
    binning="obs",
)

So if we want to change binning or time format, we first need to save the light curve data we have (if we want to keep it) and then clear the light curve data via `q.clearProduct('lightCurves')`.

Let's do that, and then demonstrate a value way of updating `q.lightCurves`

In [ ]:
q.clearProduct("lightCurves")
q.getLightCurves(byID=True, subset=mySS, timeFormat="MJD", binning="obs", bands=("total", "soft"))

Let's have a quick look at what we have:

In [ ]:
q.lightCurves.keys()

In [ ]:
q.lightCurves[209851]["Datasets"]

Now let's pretend that I forgot that I also wanted the HR1 data; I can just add these to what I already have:

In [ ]:
q.getLightCurves(byID=True, subset=mySS, timeFormat="MJD", binning="obs", bands=("HR1",))

So I should now have the total and soft data as before, and HR1 as well:

In [ ]:
q.lightCurves[209851]["Datasets"]

And I do. If I'd wanted I could also have changed the subset to get light curves for extra sources.

#### Saving light curves

You can save light curves, either by setting the `saveData=True` argument in the `getLightCurves()` call, or by running `getLightCurves()` and calling the `saveLightCurve()` function, again, these just wrap the functions in the [`data.SXSPS` module](../data/SXPS.ipynb)

Here are some quick demos:

In [ ]:
q.clearProduct("lightCurves")
q.verbose = True
q.getLightCurves(
    byID=True,
    subset=q.results["MeanOffAxisAngle"] < 2,
    timeFormat="MJD",
    binning="obs",
    bands=("total", "soft"),
    saveData=True,
    destDir="/tmp/APIDemo_SXPSq_LC1",
    subDirs=True,
)

I deliberately made the object verbose, just to show you what's being saved.

The alternative approach is to call `saveLightCurves()` separately to `getLightCurves()`. This has the advantage that we can choose to save light curves only for some of the sources, or only some light curve types, using the `whichSources` and `whichCurves` argments that `data.SXPS.saveLightCurves` takes:


In [ ]:
q.saveLightCurves(destDir="/tmp/APIDemo_SXPSq_LC2", subDirs=False, whichSources=(209851,), whichCurves=("Total_rates",))
q.verbose = False  # I want to turn this off again because it annoys me

#### Plotting light curves

You may recall the [module-level `plotLightCurve()` function](https://www.swift.ac.uk/API/ukssdc/commonFunc.md#plotlightcurve)? We can use that too. The only new argument is that we have to specify which source we want to plot; the function doesn't support plotting multiple objects (although it does return the `pyplot` objects so you could do this yourself)


In [ ]:
fig, ax = q.plotLightCurves(
    209851, whichCurves=("Total_rates", "Total_UL"), cols={"Total_rates": "red", "Total_UL": "blue"}, ylog=True
)

And having captured `fig` and `ax` if I wanted to add another dataset:

In [ ]:
fig, ax = q.plotLightCurves(
    209920,
    fig=fig,
    ax=ax,
    whichCurves=("Total_rates", "Total_UL"),
    cols={"Total_rates": "green", "Total_UL": "black"},
    ylog=True,
)
fig

<a id='sspec'></a>

### Source spectra

The situation for spectra is, amazingly enough, almost the same as for the other products we've already covered. The main thing to remind you is that for spectra, there is difference between the data returned and stored in variables, and the data saved to disk. The former gives information about the spectrum and the spectral fit results; the latter gives the actual spectral files themselves. If you want the latter then, exactly as above for light curves, you can either specify `saveData=True` during the `getSpectra()` call, or you can call `saveSpectra()` after the event. I will only demonstrate the latter here.

In [ ]:
q.getSpectra(byID=True, subset=mySS)

In [ ]:
q.spectra.keys()

In [ ]:
q.spectra[209851]

As you should, surely, have expected, this function got us a `dict` with an entry per source, indexed by ID (as requested), and these entries are themselves [spectrum `dict`s](https://www.swift.ac.uk/API/ukssdc/structures.md#the-spectrum-dict)

#### Saving the spectra

If you didn't give `saveData=True` above then you can still save the spectra later, as noted above, using the `saveSpectra()` function. As with light curves, this lets you choose to save only some of the spectra you have, if you want:

In [ ]:
q.verbose = True
q.saveSpectra(whichSources=(209851,), destDir="/tmp/APIDemo_SXPSq_spec", saveImages=True, extract=True, removeTar=True)
q.verbose = False

<a id='sim'></a>
### Source images

Source images are only saved to disk, these being just png-format images can only be saved to disk as there is no particular value (that I know of) of storing them in variables. The function thus begins `save` not `get`, and has the optional short-form `saveImages()` which will automatically redirect to `saveSourceImages()` if you have the 'sources' table selected (and `saveDatasetImages` if you have the 'datasets' table, etc.)

This only really needs one quick example:

In [ ]:
q.verbose = True  # So we see what's going on
q.saveImages(subset=mySS, byName=True, destDir="/tmp/APIDemo_SXPSq_images")

<a id='scat'></a>
#### Vizier and SIMBAD matches

The automated analysis of SXPS sources includes looking for possible counterparts in a predefined set of catalogues; you can access the results of this through the 'xcorr' table or via the 'CrossMatch' entry  in the data returned by the `getsourceDetails()`. However, you may want to look up a much wider range of catalogues, such as those available in SIMBAD and Vizier, so I've provided some basic functionality to do this. Or, more precisely, the astroquery devs have provided some time tools to use this,and I've provided some wrapper functions.

If you haven't already installed astroquery you can do that now (e.g. `pip install astroquery`), but if you do that you will probably have to restart the kernel in this notebook and redo the import of this module, otherwise it will not know that astroquery now exists.

To save you lots of scrolling, just install astroquery, restart the Jupyter kernel (select "Kernel -> Restart" from the menu) and then execute this next cell (which you can ignore if you already have astroquery). This will import the module and redo our basic source query that we're using as a basis.

In [ ]:
import swifttools.ukssdc.query as uq

q = uq.SXPSQuery(cat="LSXPS", table="sources", silent=False)
q.addConeSearch(name="GRB 060729", radius=2, units="arcmin")
q.addFilter(("DetFlag", "=", 0))
q.submit()
mySS = (q.results["MeanOffAxisAngle"] < 2) | (q.results["HR1"] > 0)

Now we also need to import astropy.units, because we'll need these for our query, as this is what `astroquery` is expecting

In [ ]:
import astropy.units as au

Right, now we're ready to actually work. There are two functions available to us:

* `doSIMBADsearch()`
* `doVizierSearch()`

and I reckon you can guess what they do.

The basic arguments are those common to everything in this module, `byID` or `byName`, `subset` and `returnData`. There are two other arguments that may be useful:

* `pandas` (default: `True`) - the `astroquery` module returns its results in the form of an astropy `Table`s, but this API has been built around pandas, so these results are converted to `DataFrame`s. You can disable this conversion by setting `pandas=False`.

These functions actually ultimately call the function `doCatSearch()` in the (abstract) base class `dataQuery` so if you want to read the docstring, you will need to do `help(uq.dataQuery.doCatSearch)`, but I'll try to make that unnecessary here.

#### SIMBAD

Let's start with a SIMBAD query.

In [ ]:
q.doSIMBADSearch(byName=True, subset=mySS, radius=20 * au.arcsec, epoch="J2000", equinox=2000)

The last 3 arguments are all things that the `astroquery.simbad.Simbad.query_region()` function needs. For more details see the help for that - essentially any arguments it needs can be passed to `doSIMBADSearch()` and will just get forwarded on.

The results of this are stored in the `q.SIMBAD` variable (and would have been returned if we said `returnData=True`. Let's explore it:

In [ ]:
q.SIMBAD.keys()

It's a `dict`, of course, and the keys are the sources we asked to query. Let's have a more detailed look at those:

In [ ]:
q.SIMBAD["LSXPS J062131.8-622213"]

This is a `DataFrame` giving the results that SIMBAD returned. For more details about the contents and how to customise them, you'll have to read up on `astroquery`.

#### Vizier

A Vizier lookup works more or less the same way, except that we don't specify the equinox and epoch. The simplest approach is to query all of Vizier **but this will be slow and you may not want to do it**. In that case just skip ahead a few cells.

In [ ]:
q.doVizierSearch(byName=True, subset=mySS, radius=20 * au.arcsec)

In [ ]:
q.Vizier.keys()

In [ ]:
q.Vizier["LSXPS J062131.8-622213"].keys()

**IF YOU SKIPPED, RESUME HERE**

As you can see from the code above (even if you didn't execute it), the results are saved in `q.Vizier`, are a `dict` with an entry per source, and then those themselves are `dict`s (if you didn't run the above, you can still infer this because I used the `.keys()` function). This `dict` is what `astroquery` returns, it is one entry for each Vizier table with a result. It's a bit much to plough through for a demonstration, and you may not have executed the above, so we'll do something simplier.

The underlying `astroquery` functionality allows us to specify a catalogue to look up in Vizier. Again, read the `astroquery` docs for full information, but to demonstrate:


In [ ]:
q.clearProduct("Vizier")  # Let's lose that mass of data we downloaded just now
q.doVizierSearch(byName=True, subset=q.results["MeanOffAxisAngle"] < 2, radius=20 * au.arcsec, catalog="GSC")

Of course, the first thing you'll notice is that while *I've* resolutely used 'cat' instead of 'catalog\[ue\]', to prevent a transatlantic incident, `astroquery` are not so kind. Pah. If it leads to hostilities, don't blame me. Anyway, what did we get?

In [ ]:
q.Vizier["LSXPS J062131.8-622213"].keys()

Looks like there were two catalogues returned that matched 'GSC'. Let's check out one of these results:

In [ ]:
q.Vizier["LSXPS J062131.8-622213"]["I/353/gsc242"]

It's a DataFrame, with all the output of the Vizier look up. Yay!

That's it for actual source products, so before I move on, let me show you one other useful function.


In [ ]:
q.clearProducts()

This has basically 'forgotten' anything we have downloaded so far, so all of the variables like `q.lightCurves`, `q.Vizier` and everything in between has been wiped.

<a id='sxpr'></a>
### XRTProductRequests

The last piece of functionality attached to the SXPS sources is the ability to make `XRTProductRequest` objects.
**PLEASE DO NOT SUBMIT HUGE NUMBERS OF THESE EN MASSE.** There are [instructions](../xrt_prods/advanced.md#scripting-large-numbers-of-jobs) showing how to submit things piecemeal, so as not to overload our servers. If you try submitting stupid number of jobs, we may have to turn on the secret death ray on Swift and fry your computer. You have been warned.

Making `XRTProductRequest` objects for SXPS sources, and all the assorted parameter options, was described in [the `swiftools.ukssdc.data.SXPS` notebook](../data/SXPS.ipynb#xpr) (you've read it now, right? Right?). I'm not repeating it all here. All you need to know for this query module is:

* This is the one function in the module which always returns what it makes, rather than saving it to a variable.
* Beyond this it works just like all the functions above.

So you only get one demo. One piece of advice though: make your query silent before proceeding; it turns out this produces loads of verbiage otherwise.

In [ ]:
q.silent = True
rlist = q.makeProductRequest(
    "MY_EMAIL_ADDRESS",
    subset=mySS,
    byID=True,
    T0="firstBlindDet",
    useObs="blind",
    addProds=["LightCurve", "Spectrum", "StandardPos"],
)

In [ ]:
rlist.keys()

OK, as expected, that returned us a `dict` with an entry for each source we asked for. Let's have a quick shufti at the results.

In [ ]:
for k, v in rlist.items():
    print(f"{k}: ")
    print(v)
    print(f"Global Pars: {v.getGlobalPars()}")
    print(f"LC Pars: {v.getLightCurvePars()}")

In [ ]:
q.getSourceDetails(byID=True, subset=q.results["MeanOffAxisAngle"] < 2)
q.sourceDetails[209920]

Looks good enough to me.

Right, that brings us to the end of the sources section, and I don't know about you, but I need a cuppa. Writing this stuff really takes a long time, and my goal of releasing this module next week is looking optimistic. So, I'm going for a drink, you can do the same if you want, and I'll be back in a few minutes to discuss datasets.

----

<a id='datasets'></a>

## Datasets

I hope you've recovered from the sources section. Datasets are much easier. First, let's get ourselves a set of datasets. I'll take GK Per, a source for which I have a soft spot, and we'll look for any datasets within 10'.

In [ ]:
q = uq.SXPSQuery(cat="LSXPS", table="datasets", silent=False)
q.addConeSearch(name="GK Per", radius=10, units="arcmin")
q.submit()

OK, 112 rows is a lot, let's make a subset for actually accessing things. Slightly annoyingly, on my screen at least, Jupyter doesn't show me the interesting columns, but I guess 'ExposureUsed' should make a worthwhile thing to filter on:

In [ ]:
q.results["ExposureUsed"].tolist()

Yeah, that's nice. I'm going to put define a subset as those with 7ks or more of used exposure, which (you can trust me on this) at the time of writing gets me 2 datasets.

In [ ]:
mySS = q.results["ExposureUsed"] > 7000

<a id='dsinfo'></a>
### Dataset Details

The first things we can retrieve are the dataset details -- the information that makes up a dataset web page. I'm feeling a bit repetitive here, and will probably give up typing this soon, but: you can read abou this functionality on the  [`data.SXPS` documentation page](../data/SXPS.ipynb#dsinfo).

As you remember from earlier, we can use either the `getDatasetDetails()` function, or just `getDetails()` and rely on the fact that our `SXPSQuery` object knows that we queried the `datasets` table. I'll just demonstrate the latter this time:

In [ ]:
q.getDetails(byObsID=True, subset=mySS)

In [ ]:
q.datasetDetails.keys()

This should not surprise you, we got two datasets, indexed by their obsID and stored in the `q.datasetDetails` variable. Each of these is a `dict` containing all the information shown in the previous notebook, you can explore yourself if you want. Note that I asked for them to be indexed by ObsID.

As with `getSourceDetails`, if we decide to get more dataset information, without first calling `q.clearProduct('datasetDetails')`, the `datasetDetails` variable will be updated. Actually, let me show something fun about this while I do it: we can change the indexing from 'ObsID' to 'DatasetID' in this new call (we could have done this for sources too). This will only affect the new objects:

In [ ]:
q.getDatasetDetails(byDatasetID=True, subset=q.results["ExposureUsed"] < 300)

q.datasetDetails.keys()

And you can see that I used `getdatasetDetails()` instead of `getDetails()` just to prove that you can.

<a id='dsim'></a>

### Dataset images

As I'm sure you remember from the [`data.SXPS` page](../data/SXPS.ipynb#sImages), or from the [source images section above](#sim), the png-format images can only be saved to disk, there being no particular value (that I know of) of storing them in variables. The function thus begins `save` not `get`, and has the optional short-form `saveImages()` which will automatically redirect to `saveDatasetImages()` if you have the `datasets` table selected.


In [ ]:
q.verbose = True  # So we see what's going on
q.saveImages(subset=mySS, byObsID=True, destDir="/tmp/APIDemo_SXPSq_images2")

----

<a id='trans'></a>

## Transients

The final table for which the `SXPSQuery` class provides enhanced functionality is the transients table.

This is going be be a *really* short section of the notebook, for the simple reason that the transient functions are the same as the source functions (except `getTransientDetails`). I mean, literally the same functions. Even if you think you're calling something else when you say `saveTransientImages()`, literally all that function does it call `saveSourceImages()`.

So, I'm going to do a query on the transients table, and then give one example per product. If you want more, then you need to scroll back up and read [the sources section](#sources).

Let's start by doing a query. I'm going to ask for all transients with the status "confirmed transient", which is Classification=1


In [ ]:
q = uq.SXPSQuery(cat="LSXPS", table="transients", silent=False)
q.addFilter(("Classification", "=", 1))
q.submit()
q.results

In [ ]:
mySS = q.results["TransientID"] < 100

I've made a quick subset too.

<a id='tinfo'></a>
### Transient Details

As you probably can guess from the introduction above, getting transient details really is as simple as calling `getTransientDetails()` or even just `getDetails()`.


In [ ]:
q.getDetails(subset=mySS)
q.transientDetails

<a id='tlc'></a>
### Transient light curves

I wonder if you can guess how we get transient light curves...

The one thing to remember (see the [`data.SXPS` transients documentation](../data/SXPS.ipynb#tlightcurves)) is that for transient light curves the available binning, timesystem and bands options are different to sources.

In [ ]:
q.getLightCurves(
    byName=True,
    subset=mySS,
    binning="counts",
)
q.lightCurves.keys()

<a id='tspec'></a>
### Transient spectra

I'm running out of vaguely humourous ways to say this, but getting transient spectra really is exactly what you should expect. Just remember that for transients you have to specify the `specType`, 'Discovery', 'Full' or 'Both'.

Purely for the sake of variation, I'll get these by ID and not bother with the subset:

In [ ]:
q.getSpectra(byID=True, specType="discovery")
q.spectra.keys()

In [ ]:
q.spectra[1]

<a id='tim'></a>
### Images

There really should be nothing I need to write here, I hope&hellip;

In [ ]:
q.verbose = True  # So we see what's going on
q.saveImages(byName=True, subset=mySS, destDir="/tmp/APIDemo_SXPSq_timages")
q.verbose = False  # So we can stop seeing what's going on

<a id='tcat'></a>

### Vizier and SIMBAD matches

I will give two examples here, one of SIMBAD and one of Vizier, but again, they really are dead straightforward if you read about sources earlier:

In [ ]:
# import astropy.units as au   # Uncomment if you didn't do this earlier
q.doSIMBADSearch(byName=True, subset=mySS, radius=20 * au.arcsec, epoch="J2000", equinox=2000)
q.SIMBAD.keys()

In [ ]:
q.SIMBAD["Swift J102732.5-643553"]

In [ ]:
q.clearProduct("Vizier")  # Let's lose that mass of data we downloaded just now
q.doVizierSearch(byName=True, radius=20 * au.arcsec, catalog="USNO-B1")
q.Vizier.keys()

In [ ]:
q.Vizier["Swift J102732.5-643553"].keys()

In [ ]:
q.Vizier["Swift J102732.5-643553"]["I/284/out"]

<a id='txpr'></a>
### XRTProductRequests

And guess what - just like for sources, we can make `XRTProductRequest` objects for transients! (And, just like sources, if you submit loads of them en masse I shall be &hellip; disappointed).

In [ ]:
q.silent = True
rlist = q.makeProductRequest(
    "MY_EMAIL_ADDRESS",
    byID=True,
    subset=mySS,
    T0="Discovery",
    useObs="new",
    addProds=[
        "LightCurve",
    ],
)

In [ ]:
for k, v in rlist.items():
    print(f"{k}: ")
    print(v)

In [ ]:
rlist[30].getLightCurvePars()

---

<a id='fulltab'></a>
### Getting the full table

One last thing which is really a kind of appendix to this module as it sort of goes against the point of it. I showed in the [`ukssdc.data.SXPS` documentation](../data/SXPS.ipynb), how you can get the entire table in one go, storing it in a `DataFrame`. I cannot see why you would want to do this for the query module, and you could always just submit an empty query I guess (although this would be slow), so being nice, I provided a quick wrapper. You literally just do: `q.getFullTable()`; the catlaogue, table and subset are properties of your `SXPSQuery` object, so you don't set them. You can provide an `epoch`, for LSXPS tables, although if you want to know what epochs are available you'll need to use the `data` module.

So, let's do one quick demo of this: I will get the full 'obsSources' table, and the result will go into the `fullTable` variable

In [ ]:
q = uq.SXPSQuery(
    cat="LSXPS",
    table="obssources",
)
q.getFullTable()
q.fullTable